### 사용자 정의 모델 클래스
- 부모 클래스: nn.Module
- 필수 오버라이딩
	* __init__(): 모델 층 구성, 설계
	* forward(): 순방향 학습 진행 코드 구현

In [1]:
import torch								# 텐서 및 수치 계산 함수 관련 모듈
import torch.nn as nn						# 인공신경망 관련 모듈
import torch.nn.functional as F 			# 손실, 거래 등 함수 관련 모듈
import torch.optim as optimizer 			# 최적화 기법 관련 모듈
from torchinfo import summary				# 모델 구조 및 정보 관련 모듈

from torchmetrics.regression import *		# 회귀 성능 지표 모듈
from torchmetrics.classification import *	# 분류 성능 지표 모듈

- [기본] 신경망 클래스
	* 입력층 - 입력 피쳐 개수로 고정
	* 출력층 - 출력 타겟 개수로 고정
	* 은닝층 - 고정

* 모델 설계
	- 데이터셋: 피쳐 4개 & 타겟 1개 , 회귀
	- 입력층  : input: 4,	output: 20		AF: ReLU
	- 은닉층  : input: 20,	output: 100  	AF: ReLU			 2진	 다중
	- 출력층  : input: 100,	output: 1 		AF: 회귀: X, 분류: (Sigmoid, Softmax)

In [16]:
# 랜덤 고정
torch.manual_seed(1)

# 텐서 저장 및 실행 위치 고정
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [17]:
class MyModel(nn.Module):
    
    # 인스턴스/객체 생성 시 자동호출 메서드
    def __init__(self, in_feature):
    
    # 부모 클래스 생성
        super().__init__()
    
    # 자식클래스 인스턴스 속성 설정
        self.input_layer = nn.Linear(in_feature, 20)		# 퍼셉트론 당 w:4개, b:1개  -> 5*20 => 100개 parameter
        self.hidden_layer = nn.Linear(20, 100)	# 퍼셉트론 당 w:20개, b:1개  -> 21*100 => 2100개 parameter
        self.output_layer = nn.Linear(100, 1)	# 퍼셉트론 당 w:100개, b:1개  -> 101*1 => 101개 parameter
    
	# Callback func
	# 		=> 순반향/전방향 학습 시, 자동호출되는 메서드
	#		=> 시스템에서 호출
	# 		=> 전달인자: Train DS
    def forward(self, x):
        print('calling forward()')
		
        y = self.input_layer(x)		# 1 Perceptron -> y = x1w1+x2w2+x3w3+x4w4 + b
									#											--> 20개
        y = F.relu(y)				# 0<= y 	--> 죽은 ReLU 발생 --> leakyReLU로 해결
		
        y = self.hidden_layer(y)	# 1 Perceptron -> y = x1w1+ ... + x20w20 + b
									# 											--> 100개
        y = F.relu(y)
		
        return self.output_layer(y)	# 1 Perceptron -> y = x1w1+ ... + x100w100 + b
        


    

In [ ]:
# 입력층의 개수가 동적인 모델

In [ ]:
# 은닉층의 개수가 동적인 모델
class MyModel4(nn.Module):
    
    # 인스턴스/객체 생성 시 자동호출 메서드
    def __init__(self,in_features, in_out, hidden_cnt, **h_out):
    
    # 부모 클래스 생성
        super().__init__()
    
    # 자식클래스 인스턴스 속성 설정
    
		if hidden_cnt == 0:	
			self.input_layer = nn.Linear(in_features, in_out)	
			self.output_layer = nn.Linear(in_out, 1)
        
		elif hidden_cnt >0:
			self.input_layer = nn.Linear(in_features, in_out)
			self.hidden_yaer = nn.Linear(in_out,h_out)
			self.output_layer = nn.Linear(h_out, 1)

		else:
			print(f'은닉층 반복횟수는 0또는 양수')
    
	# Callback func
	# 		=> 순반향/전방향 학습 시, 자동호출되는 메서드
	#		=> 시스템에서 호출
	# 		=> 전달인자: Train DS
    def forward(self, x, hidden_cnt):
        print('calling forward()')
        
        if hidden_cnt == 0:
             y = self.input_layer(x)
			 
		
        y = self.input_layer(x)		# 1 Perceptron -> y = x1w1+x2w2+x3w3+x4w4 + b
									#											--> 20개
        y = F.relu(y)				# 0<= y 	--> 죽은 ReLU 발생 --> leakyReLU로 해결
		
        y = self.hidden_layer(y)	# 1 Perceptron -> y = x1w1+ ... + x20w20 + b
									# 											--> 100개
        y = F.relu(y)
		
        return self.output_layer(y)	# 1 Perceptron -> y = x1w1+ ... + x100w100 + b

In [18]:
# 모델 인스턴스 생성
m1 = MyModel(4)

In [6]:
# for m in m1.parameters(): print(m)

In [ ]:
# for m in m1.named_parameters(): print(m)

In [19]:
# 학습 진행 ==> 모델인스턴스명(데이터)

# 임의의 데이터
dataTS = torch.FloatTensor( [[1,3,5,7], [2,4,6,8]])
targetTS = torch.FloatTensor( [[4],[5]] )

# 학습
pre_y = m1(dataTS)

print(pre_y)

calling forward()
tensor([[-0.1395],
        [-0.1858]], grad_fn=<AddmmBackward0>)
